In [1]:
# TO DO: minimise the no. of time taken as well as minimise the no. of vehicles.

# Add limit to get sublist  upto len = 4
# add cost function for vehicles. capacity * 1.2
# decide based on cost function and average time which is suitable

In [2]:
"""Capacited Vehicle Routing Problem with Pickup & Delivery and Time Window (CVRPPDTW)"""

'Capacited Vehicle Routing Problem with Pickup & Delivery and Time Window (CVRPPDTW)'

In [3]:
!pip install ortools

     |████████████████████████████████| 15.5 MB 4.6 MB/s 
     |████████████████████████████████| 1.0 MB 45.9 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.


In [4]:
!apt-get install graphviz libgraphviz-dev pkg-config

Reading package lists... Done
Building dependency tree       
Reading state information... Done
pkg-config is already the newest version (0.29.1-0ubuntu2).
graphviz is already the newest version (2.40.1-2).
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
The following additional packages will be installed:
  libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin libgtk2.0-common
  libgvc6-plugins-gtk libxdot4
Suggested packages:
  gvfs
The following NEW packages will be installed:
  libgail-common libgail18 libgraphviz-dev libgtk2.0-0 libgtk2.0-bin
  libgtk2.0-common libgvc6-plugins-gtk libxdot4
0 upgraded, 8 newly installed, 0 to remove and 42 not upgraded.
Need to get 2,120 kB of archives.
After this operation, 7,128 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libgtk2.0-common all 2.24.32-1ubuntu1 [125 kB]
Get:2 http://

In [5]:
!pip install pygraphviz

     |████████████████████████████████| 118 kB 4.3 MB/s 
  Created wheel for pygraphviz: filename=pygraphviz-1.7-cp37-cp37m-linux_x86_64.whl size=165752 sha256=37b57c2737df4c00214e5f0dc3aa3475a2635a1e9e47fab6d903571dcdd7e8af
  Stored in directory: /root/.cache/pip/wheels/8c/bc/0c/ac35392b72556e75107ff610cb31b313e8471918a6d280e34c
Successfully built pygraphviz


In [6]:
from typing import Callable

from itertools import combinations
import pygraphviz as pgv
import yaml
from ortools.constraint_solver.pywrapcp import (
    Assignment,
    DefaultRoutingSearchParameters,
    RoutingDimension,
    RoutingIndexManager,
    RoutingModel,
)
from ortools.constraint_solver.routing_enums_pb2 import (
    FirstSolutionStrategy,
    LocalSearchMetaheuristic,
)

In [7]:
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    # Weights (i.e costs, such as time) between nodes
    # For example, cost 6 at index (0, 1) represents a cost between node 0 and 1 is 6
    data['weights'] = [
        [0,  6,  9,  8,  6,  3,  6,  2,  3,  2,  6,  6,  4,  4,  5,  9,  7],
        [6,  0,  8,  3,  2,  6,  8,  4,  8,  8, 13,  7,  5,  8, 12, 10, 14],
        [9,  8,  0, 11, 10,  6,  3,  9,  5,  8,  4, 15, 13, 13,  9, 18,  9],
        [8,  3, 11,  0,  1,  7, 10,  6, 10, 10, 14,  6,  7,  9, 13,  6, 16],
        [6,  2, 10,  1,  0,  6,  9,  4,  8,  9, 13,  4,  6,  8, 12,  8, 14],
        [3,  6,  6,  7,  6,  0,  2,  3,  2,  2,  6,  9,  7,  7,  6, 12,  8],
        [6,  8,  3, 10,  9,  2,  0,  6,  2,  5,  4, 12, 10, 10,  6, 15,  5],
        [2,  4,  9,  6,  4,  3,  6,  0,  4,  4,  8,  5,  4,  3,  7,  8, 10],
        [3,  8,  5, 10,  8,  2,  2,  4,  0,  3,  4,  9,  8,  7,  3, 13,  6],
        [2,  8,  8, 10,  9,  2,  5,  4,  3,  0,  4,  6,  5,  4,  3,  9,  5],
        [6, 13,  4, 14, 13,  6,  4,  8,  4,  4,  0, 10,  9,  8,  4, 13,  4],
        [6,  7, 15,  6,  4,  9, 12,  5,  9,  6, 10,  0,  1,  3,  7,  3, 10],
        [4,  5, 13,  7,  6,  7, 10,  4,  8,  5,  9,  1,  0,  2,  6,  4,  8],
        [4,  8, 13,  9,  8,  7, 10,  3,  7,  4,  8,  3,  2,  0,  4,  5,  6],
        [5, 12,  9, 13, 12,  6,  6,  7,  3,  3,  4,  7,  6,  4,  0,  9,  2],
        [9, 10, 18,  6,  8, 12, 15,  8, 13,  9, 13,  3,  4,  5,  9,  0,  9],
        [7, 14,  9, 16, 14,  8,  5, 10,  6,  5,  4, 10,  8,  6,  2,  9,  0]
      ]

    # Times it takes for a vehicle to leave from the time it stops at each node
    data['service_times'] = [0, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]

    # Time windows for nodes which are considered as requested times for a visit
    # Vehicles must visit a node within its time window
    data['time_windows'] = [
    [ 0,   0],  # 0
    [55,  75],  # 1
    [75,  95],  # 2
    [60,  80],  # 3
    [35,  55],  # 4
    [ 0,  20],  # 5
    [55,  75],  # 6
    [ 0,  20],  # 7
    [10,  30],  # 8
    [ 0,  20],  # 9
    [75,  95],  # 10
    [85, 105],  # 11
    [ 5,  25],  # 12
    [15,  35],  # 13
    [10,  30],  # 14
    [45,  65],  # 15
    [30,  50]   # 16
    ]
    
    data['vol_prod_1'] = 10      # Volume of single piece of product 1
    data['vol_prod_2'] = 5       # Volume of single piece of product 2

    # Demands corresponding to the volume of items to be picked up
    # data['demands'] = [0, 1, 1, 2, 3, 2, 3, 3, 1, 1, 2, 1, 2, 3, 3, 1, 1]
    data['demands_1'] = [0, 10, 10, 20, 40, 20, 40, 80, 80, 10, 20, 10, 20, 40, 40, 80, 80]           # Demand of product 1 at each location
    data['demands_2'] = [0, 5, 10, 25, 0, 20, 35, 15, 5, 40, 45, 20, 30, 0, 25, 0, 10]                # Demand of product 2 at each location
    data['total_demands'] = [a + b for a, b in zip(data['demands_1'], data['demands_2'])]             # Total demand at each location
    # Demands considering 
        # pickup locations also have demands: [0, 1, 1, 2, 4, 2, 4, 8, 8, 1, 2, 1, 2, 4, 4, 8, 8]
        # pickup locations have 0 demand: [0, 3, 0, 0, 12, 0, 12, 0, 0, 3, 3, 9, 6, 0, 0, 0, 12]

    # Pickup and Delivery locations. The first index in each list is pickup location
    # and second index is the delivery location
    data['pickups_deliveries'] = [
        [8, 6],
        [2, 10],
        [7, 4],
        [5, 9],
        [3, 1],
        [15, 11],
        [13, 12],
        [14, 16],
    ]

    # Total No. of locations
    data['num_locations'] = 17

    # Total No. of vehicles
    data['num_vehicles'] = 10

    # The maximum time per vehicle
    data['max_time'] = 100

    # Capacity of the vehicles
    data['vehicle_capacities'] = [300, 350, 450, 200, 250, 400, 150, 100, 275, 175]

    # Index of the start node
    data['depot'] = 0

    return data

In [8]:
def create_weight_callback(manager: RoutingIndexManager, data: dict):
    """
    Create a callback to return the weight between points.
    """
    def weight_callback(from_index: int, to_index: int) -> int:
        """
        Return the weight between the two points.
        """
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["weights"][from_node][to_node]
    return weight_callback

In [9]:
def create_demand_callback(manager: RoutingIndexManager, data: dict):
    """
    Create a callback to get demands at each location.
    """
    def demand_callback(from_index: int) -> int:
        """
        Return the demand.
        """
        from_node = manager.IndexToNode(from_index)
        return data["total_demands"][from_node]

    return demand_callback

In [10]:
def create_time_callback(manager: RoutingIndexManager, data: dict):
    """
    Create a callback to get total times between locations.
    """
    def time_callback(from_index: int, to_index: int) -> int:
        """
        Return the total time between the two nodes.
        """
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)

        # Get the service time to the specified location
        serv_time = data["service_times"][from_node]
        # Get the travel times between two locations
        trav_time = data["weights"][from_node][to_node]

        return serv_time + trav_time

    return time_callback

In [11]:
def add_capacity_constraints(
    routing: RoutingModel,
    manager: RoutingIndexManager,
    data: dict,
    demand_callback_index: int,
    vehicle_cap: list,
):
    """
    Add capacity constraints.
    """
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        slack_max=0,  # null capacity slack
        vehicle_capacities=vehicle_cap,  # vehicle maximum capacities
        fix_start_cumul_to_zero=True,  # start cumul to zero
        name="Capacity",
    )

In [12]:
def add_time_window_constraints(
    routing: RoutingModel,
    manager: RoutingIndexManager,
    data: dict,
    time_callback_index: int,
):
    """
    Add time window constraints.
    """
    horizon = data["max_time"]
    routing.AddDimension(
        time_callback_index,
        slack_max=horizon,  # allow waiting time
        capacity=horizon,  # maximum time per vehicle
        # Don't force start cumul to zero. This doesn't have any effect in this example,
        # since the depot has a start window of (0, 0).
        fix_start_cumul_to_zero=False,
        name="Time",
    )
    time_dimension = routing.GetDimensionOrDie("Time")
    for loc_idx, (open_time, close_time) in enumerate(data["time_windows"]):
        index = manager.NodeToIndex(loc_idx)
        time_dimension.CumulVar(index).SetRange(open_time, close_time)

    # Add pickup and delivery requests, using pickup and delivery locations in data['pickups_deliveries']
    for request in data['pickups_deliveries']:
        pickup_index = manager.NodeToIndex(request[0])
        delivery_index = manager.NodeToIndex(request[1])
        # Adding request
        routing.AddPickupAndDelivery(pickup_index, delivery_index)
        # To ensure that each item must be picked up and delivered by the same vehicle
        routing.solver().Add(
            routing.VehicleVar(pickup_index) == routing.VehicleVar(
                delivery_index))
        # To ensure that each item must be picked up before it is delivered
        routing.solver().Add(
            time_dimension.CumulVar(pickup_index) <=
            time_dimension.CumulVar(delivery_index))

In [13]:
def print_solution(
    data: dict,
    routing: RoutingModel,
    manager: RoutingIndexManager,
    assignment: Assignment,
    vehicle_list: list,
):
    """
    Print the solution.
    """
    capacity_dimension = routing.GetDimensionOrDie("Capacity")
    time_dimension = routing.GetDimensionOrDie("Time")
    total_time = 0
    consider_vehicle_list = True
    for vehicle_id in range(len(vehicle_list)):
        index = routing.Start(vehicle_id)
        node_props = []

        while not routing.IsEnd(index):
            props = node_properties(
                manager, assignment, capacity_dimension, time_dimension, index
            )
            node_props.append(props)
            index = assignment.Value(routing.NextVar(index))

        props = node_properties(
            manager, assignment, capacity_dimension, time_dimension, index
        )
        node_props.append(props)
        route_time = assignment.Value(time_dimension.CumulVar(index))

        # For any of the vehicle in the fleet, if its route_time is 0 then
        # we will skip the respective fleet of the vehicles.
        if route_time == 0:
          consider_vehicle_list = False
          break
        route = "\n  -> ".join(
            ["[Node %2s: Load(%s) Time(%2s, %s)]" % prop for prop in node_props]
        )
        plan_output = (
            f"Route for vehicle {vehicle_id}:\n  {route}\n"
            f"Load of the route: {props[1]}\nTime of the route: {route_time} min\n"
        )
        print(plan_output)
        total_time += route_time

    if not consider_vehicle_list:
      return
    print(f"Total time of all routes: {total_time} min")
    return total_time

In [14]:
def node_properties(
    manager: RoutingIndexManager,
    assignment: Assignment,
    capacity_dimension: RoutingDimension,
    time_dimension: RoutingDimension,
    index: int,
) -> tuple:
    """
    Get a node's properties corresponding to the index.
    """
    node_index = manager.IndexToNode(index)
    load = assignment.Value(capacity_dimension.CumulVar(index))
    time_var = time_dimension.CumulVar(index)
    time_min, time_max = assignment.Min(time_var), assignment.Max(time_var)
    return (node_index, load, time_min, time_max)

In [15]:
def draw_route_graph(
    filename: str,
    data: dict,
    routing: RoutingModel,
    manager: RoutingIndexManager,
    assignment: Assignment,
    vehicle_list: list,
    prog="sfdp",
):
    """
    Draw a route graph based on the solution of the problem.
    """
    weights = data["weights"]
    demands = data["total_demands"]
    time_windows = data["time_windows"]
    graph = pgv.AGraph(directed=True)

    def _node(index: int) -> str:
        if index == 0:
            return f"{index}\nDepot"
        return f"{index}\nDemand: {demands[index]}\nRange: {time_windows[index]}"

    for vehicle_id in range(len(vehicle_list)):
        index = routing.Start(vehicle_id)
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            next_index = assignment.Value(routing.NextVar(index))
            next_node_index = manager.IndexToNode(next_index)
            weight = weights[node_index][next_node_index]
            graph.add_edge(
                _node(node_index), _node(next_node_index), weight=weight, label=weight
            )
            index = next_index

    graph.draw(filename, prog=prog)

    print(f"The route graph has been saved to {filename}.")

In [16]:
def find_sublist(list_of_vehicles, total_demand):
    """
    This function finds the different combinations of vehicles who can fulfill the total demand
    """
    capacity_lists = []
    for i in range(len(list_of_vehicles)):
        sum_so_far = 0
        # consider all sublists starting from `i` and ending at `j`
        for j in range(i, len(list_of_vehicles)):
            # target of elements so far
            sum_so_far += list_of_vehicles[j]
            # if the sum so far is equal to the given sum
            if sum_so_far >= total_demand and sum_so_far < (total_demand * 1.5):
                capacity_lists.append(list_of_vehicles[i:j+1])
    return capacity_lists

In [17]:
def find_capacity_sublist(list_of_vehicles, total_demand):
    """
    This function finds the different combinations of vehicles who can fulfill the total demand
    """
    all_capacity_lists = []
    # Iterating through all possible
    # subsets of arr from lengths 0 to n:
    for i in range(len(list_of_vehicles)+1):
        for subset in combinations(list_of_vehicles, i):
              
            # printing the subset if its sum is x:
            if sum(subset) >= total_demand and sum(subset) < (total_demand * 1.5):
                # print(list(subset))
                all_capacity_lists.append(list(subset))

    # Remove the duplicate sublists
    capacity_lists, capacity_set = [], set()
    for sublist in all_capacity_lists:
        sorted_sublist = tuple(sorted(sublist))
        if not sorted_sublist in capacity_set:
            capacity_lists.append(sublist)
            capacity_set.add(sorted_sublist)
    return capacity_lists

In [18]:
def find_cost_of_vehicle(list_of_vehicle_capacity):
    """
    This function calculates the total cost of the fleet of vehicles
    Here we are assuming that,
      cost_of_each_vehicle = capacity_of_vehicle * 100
    """
    cost = 0
    for capacity in list_of_vehicle_capacity:
      cost += capacity*100
    return cost

In [19]:
def print_avg_time_and_cost(average_journey_time, cost_of_the_fleet):
    """
    This function prints the average_journey_time and cost of the fleet
    """
    print("The average time of all the possible fleets is \n")
    for key, value in average_journey_time.items():
      print(key, ' : ', value)
    print("\nThe cost of each fleet calculated based on the capacity of vehicles is \n")
    for key, value in cost_of_the_fleet.items():
      print(key, ' : ', value)
    key_list = list(cost_of_the_fleet.keys())
    val_list = list(cost_of_the_fleet.values())
    min_cost = min(list(cost_of_the_fleet.values()))
    index_of_min_cost = val_list.index(min_cost)
    key_for_min_cost = key_list[index_of_min_cost]
    print(f"The fleet of the vehicle with minimum cost is {key_for_min_cost} with cost {min_cost}")

In [20]:
def main():
    """
    Entry point of the program.
    """
    # Instantiate the data problem
    data = create_data_model()

    total_demand = sum(data['total_demands'])
    vehicle_capacity_list = find_sublist(data['vehicle_capacities'], total_demand)
    # Sort the list based on the length of sublists
    vehicle_capacity_list.sort(key=len)

    # vehicle_capacity_list = find_capacity_sublist(data['vehicle_capacities'], total_demand)
    print(f"For given total demand {total_demand}, we have {len(vehicle_capacity_list)} different fleets of vehicles with different capacities: \n{vehicle_capacity_list}")
    print("="*100)

    average_journey_time = {}
    cost_of_the_fleet = {}
    for sublist in vehicle_capacity_list:
      print(f"Using {len(sublist)} vehicles with capacities: {sublist}")

      manager = RoutingIndexManager(
          data["num_locations"], len(sublist), data["depot"]
      )
      routing = RoutingModel(manager)

      # Define weights of edges
      weight_callback_index = routing.RegisterTransitCallback(
          create_weight_callback(manager, data)
      )
      routing.SetArcCostEvaluatorOfAllVehicles(weight_callback_index)

      # Add capacity constraints
      demand_callback = create_demand_callback(manager, data)
      demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
      add_capacity_constraints(routing, manager, data, demand_callback_index, sublist)

      # Add time window constraints
      time_callback_index = routing.RegisterTransitCallback(
          create_time_callback(manager, data)
      )
      add_time_window_constraints(routing, manager, data, time_callback_index)

      # Set first solution heuristic (cheapest addition)
      search_params = DefaultRoutingSearchParameters()
      search_params.first_solution_strategy = FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION
      search_params.local_search_metaheuristic = (
          LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
      )
      # NOTE: Since Guided Local Search could take a very long time, we set a
      # reasonable time limit
      search_params.time_limit.seconds = 30

      # Solve the problem
      assignment = routing.SolveWithParameters(search_params)

      if not assignment:
          print("No solution found.")
          continue
      journey_time = print_solution(data, routing, manager, assignment, sublist)
      if journey_time:
        average_journey_time[f"{sublist}"] = journey_time/len(sublist)
        cost_of_the_fleet[f"{sublist}"] = find_cost_of_vehicle(sublist)

        # Export route graphs
        draw_route_graph(f"/content/route_{sublist}.png", data, routing, manager, assignment, sublist)
        print("="*100)
      else:
        continue

    print_avg_time_and_cost(average_journey_time, cost_of_the_fleet)

In [21]:
if __name__ == "__main__":
    main()

For given total demand 885, we have 12 different fleets of vehicles with different capacities: 
[[300, 350, 450], [350, 450, 200], [450, 200, 250], [300, 350, 450, 200], [350, 450, 200, 250], [450, 200, 250, 400], [200, 250, 400, 150], [250, 400, 150, 100], [400, 150, 100, 275], [200, 250, 400, 150, 100], [250, 400, 150, 100, 275], [400, 150, 100, 275, 175]]
Using 3 vehicles with capacities: [300, 350, 450]
No solution found.
Using 3 vehicles with capacities: [350, 450, 200]
Route for vehicle 0:
  [Node  0: Load(0) Time( 0, 0)]
  -> [Node  5: Load(0) Time( 3, 13)]
  -> [Node  9: Load(40) Time(10, 20)]
  -> [Node  8: Load(90) Time(18, 30)]
  -> [Node  6: Load(175) Time(55, 72)]
  -> [Node  2: Load(250) Time(75, 80)]
  -> [Node 10: Load(270) Time(84, 89)]
  -> [Node  0: Load(335) Time(95, 100)]
Load of the route: 335
Time of the route: 95 min

Route for vehicle 1:
  [Node  0: Load(0) Time( 0, 8)]
  -> [Node  7: Load(0) Time( 2, 10)]
  -> [Node 13: Load(95) Time(15, 18)]
  -> [Node 12: Lo

Assignment is of type ortools.constraint_solver.pywrapcp.Assignment class.

We get the assignment with vehicles assigned for proper routes.

vrpy has a flag : use_all_vehicles (bool, optional) – True if all vehicles specified by num_vehicles should be used. Defaults to False.

Checked with different first solution strategy. They are explained at:
https://developers.google.com/optimization/routing/routing_options